In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

In [2]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x**2*y+y+2

In [3]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

print('Result: ', result)

Result:  42


Instead of manually running the initializer for every single variable, we can use the `global_variables_initializer()` function. Note that it does not actually perform the initialization immediately, but rather creates a node in the graph that will initialize all variables when it is run.

In [4]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

print('Result: ', result)

Result:  42


## Managing graphs

Any node we create is automatically added to the default graph

In [5]:
x1 = tf.Variable(1, name='x1')
x1.graph is tf.get_default_graph()

True

Sometimes we may want to manage multiple independent graphs. We can do this by creating a new Graph and temporarily making it the default graph inside a `with` block.

In [6]:
graph = tf.Graph()

with graph.as_default():
    x2 = tf.Variable(2, name='x2')

x2.graph is tf.get_default_graph()

False

## Linear regression demo with normal equation

In [7]:
housing = load_boston()
print(housing.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [8]:
(m, n) = housing.data.shape
housing_data_including_bias = np.hstack((np.ones((m, 1)), housing.data))

In [9]:
X = tf.constant(housing_data_including_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
X_T = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(X_T, X)), X_T), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [10]:
theta_value

array([[ 3.6453293e+01],
       [-1.0800815e-01],
       [ 4.6423875e-02],
       [ 2.0537592e-02],
       [ 2.6874626e+00],
       [-1.7761471e+01],
       [ 3.8099995e+00],
       [ 6.8512064e-04],
       [-1.4755175e+00],
       [ 3.0603766e-01],
       [-1.2334908e-02],
       [-9.5264518e-01],
       [ 9.3140695e-03],
       [-5.2475297e-01]], dtype=float32)

## Linear regression demo with gradient descent

In [11]:
standard_scaler = StandardScaler()
housing_data_including_bias_scaled = standard_scaler.fit_transform(
    housing_data_including_bias)

In [14]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_including_bias_scaled, dtype=tf.float32, name='X')
X_T=tf.matrix_transpose(X)
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')

theta = tf.Variable(tf.random_uniform(
    [n+1, 1], minval=-1.0, maxval=1.0), dtype=tf.float32, name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred-y
mse = tf.reduce_mean(tf.square(error),name='mse')
# gradients=(2/m)*tf.matmul(X_T,error)
gradients = tf.gradients(mse, [theta])[0]


training_op=tf.assign(theta,theta-(learning_rate*gradients))

init=tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print('Epoch ',epoch,' MSE = ',mse.eval())
        sess.run(training_op)
        
    best_theta=theta.eval()
    
print('Theta: ',best_theta)

Epoch  0  MSE =  582.1076
Epoch  100  MSE =  531.32227
Epoch  200  MSE =  530.49817
Epoch  300  MSE =  530.1771
Epoch  400  MSE =  529.9987
Epoch  500  MSE =  529.88916
Epoch  600  MSE =  529.81714
Epoch  700  MSE =  529.76746
Epoch  800  MSE =  529.732
Epoch  900  MSE =  529.706
Theta:  [[ 0.5273626 ]
 [-0.8723843 ]
 [ 0.98486894]
 [-0.10348937]
 [ 0.71688664]
 [-1.9553572 ]
 [ 2.7370389 ]
 [-0.02864742]
 [-3.069595  ]
 [ 2.0207326 ]
 [-1.3738025 ]
 [-2.0270154 ]
 [ 0.8490067 ]
 [-3.7134821 ]]
